## Download Data from Google-Drive

In [1]:
'''
File Name : Data
File Link : https://drive.google.com/file/d/1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw/view?usp=share_link
File Id : '1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw'

'''

!gdown --id 1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw
!unzip ccle_ctrpv2_gdse.zip

'''
File Name : L1000
File Link : https://drive.google.com/file/d/1caDHIUWEFZkZsoGBOg9dImNFQuxI7k8K/view?usp=sharing
File Id : '1caDHIUWEFZkZsoGBOg9dImNFQuxI7k8K'

'''

!gdown --id 1caDHIUWEFZkZsoGBOg9dImNFQuxI7k8K

/bin/bash: /home/amirhosein/anaconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/amirhosein/anaconda3/envs/tf/lib/python3.9/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw
From (redirected): https://drive.google.com/uc?id=1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw&confirm=t&uuid=8ca9fd5b-7d42-4718-86f4-e2f69879b231
To: /home/amirhosein/Documents/Codes/Drug Response Prediction/ccle_ctrpv2_gdse.zip
  3%|█▏                                     | 16.3M/557M [00:08<03:28, 2.59MB/s]^C
Traceback (most recent call last):
  File "/home/amirhosein/anaconda3/envs/tf/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/home/amirhosein/anaconda3/envs/tf/lib/python3.9/site-packages/gdown/cli.py", line 156, in main
    f

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error , mean_absolute_error
from time import perf_counter, sleep
import math
import scipy.stats as stats
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

## Read train and test data

In [2]:
# x_train
ccle_ctrpv2 = pd.read_csv("../data/ccle_ctrpv2_rnaseq_tpm.csv")
# x_test
ccle_ctrpv2_aac = pd.read_csv("../data/ccle_ctrpv2_aac.csv")
# y_train
gdse_rnaseq = pd.read_csv("../data/gdse_rnaseq_tpm.csv")
# y_test
gdse_aac = pd.read_csv("../data/gdse_aac.csv")
# L1000 
L1000f = pd.read_csv("../data/L1000_gene_list.txt", delimiter = "\t")['Symbol']

## Cleaning the dataframes

In [3]:
removing_features = ['PAPD7', 'HDGFRP3', 'AARS', 'TMEM2', 'TMEM5', 'SQRDL', 'H2AFV', 'KIAA0907', 'SACM1L', 'HIST2H2BE', 'IKBKAP', 'TMEM110', 'WRB', 'FAM69A', 'FAM57A', 'ATP5S', 'NARFL', 'KIF1BP', 'HN1L', 'EPRS', 'HIST1H2BK']

L1000f = [item for item in L1000f if item not in removing_features]

drug_names = gdse_aac.columns.values.tolist()
drug_names = drug_names[1:]


ccle_ctrpv2 = ccle_ctrpv2.rename(columns={"Unnamed: 0": "sample"})
gdse_rnaseq = gdse_rnaseq.rename(columns={"Unnamed: 0": "sample"})
ccle_ctrpv2_aac = ccle_ctrpv2_aac.rename(columns={"Unnamed: 0": "sample"})
gdse_aac = gdse_aac.rename(columns={"Unnamed: 0": "sample"})

In [4]:
def remove_nan(data):
    data = data.dropna()
    data.reset_index(drop = True)
    return data

In [5]:
'''
Initializing k-fold cross-validation.

n_splits=4: the dataset will be split into 4 equal-sized folds or subsets.

shuffle: data will be shuffled before splitting into folds, 
         ensuring randomization in the distribution of samples across each fold.

'''
kfold = KFold(n_splits=4, shuffle=True)


'''
Initializing StandardScaler to normalize the data to have mean of 0 and variance of 1.
'''
Sscaler = StandardScaler()

## Creating a shallow neural network
---
1. **Layers**

First Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Second Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Third Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Fourth Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>

Output Layer -> **Activation Function:** *Sigmoid* <br>

Sigmoid activation function is used to force the output to have a range between 0 and 1, 
as the desired output (Drug Response) has a value between 0 and 1.

2. **Optimizer**

Using *Stochastic Gradient descent* (SGD) with momentum:

        learning rate: 0.01
        momentum = 0.9

3. **Loss Function**

Using *mean squared error* (MSE) for the loss function

4. **Regularization technique**

20% dropout after each hidden layer 

5. **Network schematic**

![MarineGEO circle logo](Arch.png)

In [6]:
model = tf.keras.models.Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1 , activation='sigmoid'))

learning_rate = 0.01
momentum = 0.9
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate, momentum)

optimizer_rmsprop = tf.keras.optimizers.RMSprop()

# Training procedure

*For each drug in the drug list:*
   1. Split the data in to train and test sets, and then remove the Nan values.
   2. Merging X_train with ytrain, and X_test with  y_test because the samples in each are different.
   3. Normalize the training X. </br>
   
   *Running the following 6 times:*
   
       4. Applying 4-fold Cross Valisation
       5. Applying the relevant features selection technique, and extract the selected features.
       6. Train the model based on the CV data and extracted features
       7. Saving the best model performed in CV and the best features
       
   8. Predicting the test set using the best model from the previous section
   9. Calculating the correlation score between the predicted drug_respons and the actual drug_response

In [7]:
cor_df = pd.DataFrame([], columns = ['corr'])

correlatoins = []
correlatoins = []
mse_train_list = []
rmse_train_list = []
mae_train_list = []
spearmanr_correlation_train_list = []
kendalltau_correlation_train_list = []
cr = []
mse = []
rmse = []
mae = []
spearmanr =[]
kendalltau = []
start = perf_counter()
for drug in drug_names:
    
    selected_features = []
    best_model_top_features = set()
    model_performances = []
    model_acc = []
    pearson_correlations = []
    

    '''
        train test split
    '''
    
    y_train = ccle_ctrpv2_aac[['sample', drug]]
    y_test  = gdse_aac[['sample', drug]]

    y_train = remove_nan(y_train)
    y_test  = remove_nan(y_test)

    ccle_ctrpv2 = remove_nan(ccle_ctrpv2)
    gdse_rnaseq = remove_nan(gdse_rnaseq)


    '''
    Merge
    '''

    gdse_rnaseq_merged_df = pd.merge(gdse_rnaseq, y_test, on='sample', how='inner')
    ccle_ctrpv2_merged_df = pd.merge(ccle_ctrpv2, y_train, on='sample', how='inner')

    X_train = ccle_ctrpv2_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]
    X_test  = gdse_rnaseq_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]

    Y_train = ccle_ctrpv2_merged_df.iloc[:, len(gdse_rnaseq.columns):]
    Y_test  = gdse_rnaseq_merged_df.iloc[:, len(gdse_rnaseq.columns):]

    X_train_norm = Sscaler.fit_transform(X_train)
    X_train_normalized = pd.DataFrame(X_train_norm, columns=X_train.columns)
    
    for i in range(6):
            
        selected_features = []
        best_model_top_features = set()
        model_performances = []
        model_acc = []
        pearson_correlations = []
        best_model_path = 'saved_models/best_model_'+ drug + str(i)
        
        
        for train, validation in kfold.split(X_train_normalized , Y_train):

            train_data = X_train_normalized.iloc[train]
            train_labels = Y_train.iloc[train]
            val_data = X_train_normalized.iloc[validation]
            val_labels = Y_train.iloc[validation]



            top_features = np.asarray(L1000f)


            train_data_selected = train_data[top_features]
            val_data_selected = val_data[top_features]


            model.compile(optimizer = optimizer_sgd, loss='mean_squared_error', metrics='MAE')

            checkpoint_callback = ModelCheckpoint(filepath=best_model_path+ '.h5' , monitor='MAE', save_best_only=True, mode='min')

            history = model.fit(train_data_selected, train_labels, epochs=15,callbacks=[checkpoint_callback], validation_data=(val_data_selected, val_labels))


        new_X = X_test[top_features]

        Xtest_norm = Sscaler.fit_transform(new_X)
        Xtest_normalized = pd.DataFrame(Xtest_norm, columns=new_X.columns)

        loaded_model = tf.keras.models.load_model(best_model_path+ '.h5' )

        gdsc_predictions = loaded_model.predict(Xtest_normalized)

        Y_test_array = np.asarray(Y_test)

        correlation, _ = pearsonr(Y_test_array.flatten(),  gdsc_predictions.flatten())
        mse_train = mean_squared_error(Y_test_array.flatten(), gdsc_predictions.flatten())
        rmse_train = math.sqrt(mse_train)
        mae_train = mean_absolute_error(Y_test_array.flatten(), gdsc_predictions.flatten())

    # Calculate Correlation for the validation data
        spearmanr_correlation_train, _ = stats.spearmanr(Y_test_array.flatten(), gdsc_predictions.flatten())
        kendalltau_correlation_train, _ = stats.kendalltau(Y_test_array.flatten(), gdsc_predictions.flatten())
    
        correlatoins.append(correlation)
        mse_train_list.append(mse_train)
        rmse_train_list.append(rmse_train)
        mae_train_list.append(mae_train)
        spearmanr_correlation_train_list.append(spearmanr_correlation_train)
        kendalltau_correlation_train_list.append(kendalltau_correlation_train)
        
        
        
    index = correlatoins.index(max(correlatoins))
    cr.append(max(correlatoins))
    mse.append(mse_train_list[index])
    rmse.append(rmse_train_list[index])
    mae.append(mae_train_list[index])
    spearmanr.append(spearmanr_correlation_train_list[index])
    kendalltau.append(kendalltau_correlation_train_list[index])
    correlatoins = []
    mse_train_list = []
    rmse_train_list = []
    mae_train_list = []
    spearmanr_correlation_train_list = []
    kendalltau_correlation_train_list = []
    
    
    
end = perf_counter()

Epoch 1/15
18/18 [==============================] - 1s 19ms/step - loss: 0.0552 - MAE: 0.1904 - val_loss: 0.0126 - val_MAE: 0.0843
Epoch 2/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0186 - MAE: 0.1108 - val_loss: 0.0130 - val_MAE: 0.0876
Epoch 3/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0147 - MAE: 0.0986 - val_loss: 0.0068 - val_MAE: 0.0628
Epoch 4/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0140 - MAE: 0.0935 - val_loss: 0.0065 - val_MAE: 0.0625
Epoch 5/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0116 - MAE: 0.0852 - val_loss: 0.0061 - val_MAE: 0.0593
Epoch 6/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0112 - MAE: 0.0851 - val_loss: 0.0058 - val_MAE: 0.0579
Epoch 7/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0102 - MAE: 0.0789 - val_loss: 0.0060 - val_MAE: 0.0591
Epoch 8/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0107 - MAE: 0.0

18/18 [==============================] - 0s 7ms/step - loss: 0.0048 - MAE: 0.0540 - val_loss: 0.0033 - val_MAE: 0.0453
Epoch 4/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0050 - MAE: 0.0544 - val_loss: 0.0032 - val_MAE: 0.0450
Epoch 5/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0053 - MAE: 0.0569 - val_loss: 0.0033 - val_MAE: 0.0453
Epoch 6/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0051 - MAE: 0.0555 - val_loss: 0.0033 - val_MAE: 0.0458
Epoch 7/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0052 - MAE: 0.0559 - val_loss: 0.0034 - val_MAE: 0.0460
Epoch 8/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0049 - MAE: 0.0541 - val_loss: 0.0034 - val_MAE: 0.0459
Epoch 9/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0047 - MAE: 0.0523 - val_loss: 0.0033 - val_MAE: 0.0455
Epoch 10/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0050 - MAE: 0.0554 - val_l

18/18 [==============================] - 0s 5ms/step - loss: 0.0040 - MAE: 0.0487 - val_loss: 0.0033 - val_MAE: 0.0466
Epoch 6/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0040 - MAE: 0.0490 - val_loss: 0.0034 - val_MAE: 0.0476
Epoch 7/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0038 - MAE: 0.0485 - val_loss: 0.0035 - val_MAE: 0.0489
Epoch 8/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0039 - MAE: 0.0482 - val_loss: 0.0034 - val_MAE: 0.0483
Epoch 9/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0037 - MAE: 0.0468 - val_loss: 0.0034 - val_MAE: 0.0480
Epoch 10/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0035 - MAE: 0.0452 - val_loss: 0.0034 - val_MAE: 0.0477
Epoch 11/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0037 - MAE: 0.0469 - val_loss: 0.0033 - val_MAE: 0.0473
Epoch 12/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0036 - MAE: 0.0464 - val

18/18 [==============================] - 0s 5ms/step - loss: 0.0031 - MAE: 0.0443 - val_loss: 0.0024 - val_MAE: 0.0394
Epoch 8/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0030 - MAE: 0.0424 - val_loss: 0.0024 - val_MAE: 0.0394
Epoch 9/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0029 - MAE: 0.0416 - val_loss: 0.0024 - val_MAE: 0.0396
Epoch 10/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0030 - MAE: 0.0428 - val_loss: 0.0023 - val_MAE: 0.0387
Epoch 11/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0031 - MAE: 0.0433 - val_loss: 0.0023 - val_MAE: 0.0392
Epoch 12/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0029 - MAE: 0.0419 - val_loss: 0.0024 - val_MAE: 0.0396
Epoch 13/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0028 - MAE: 0.0408 - val_loss: 0.0024 - val_MAE: 0.0396
Epoch 14/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0029 - MAE: 0.0429 - v

18/18 [==============================] - 0s 4ms/step - loss: 0.0027 - MAE: 0.0403 - val_loss: 0.0020 - val_MAE: 0.0373
Epoch 10/15
18/18 [==============================] - 0s 4ms/step - loss: 0.0026 - MAE: 0.0390 - val_loss: 0.0020 - val_MAE: 0.0372
Epoch 11/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0026 - MAE: 0.0398 - val_loss: 0.0020 - val_MAE: 0.0376
Epoch 12/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0028 - MAE: 0.0414 - val_loss: 0.0020 - val_MAE: 0.0377
Epoch 13/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0024 - MAE: 0.0386 - val_loss: 0.0020 - val_MAE: 0.0377
Epoch 14/15
18/18 [==============================] - 0s 5ms/step - loss: 0.0028 - MAE: 0.0401 - val_loss: 0.0020 - val_MAE: 0.0376
Epoch 15/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0025 - MAE: 0.0383 - val_loss: 0.0020 - val_MAE: 0.0378
Epoch 1/15
18/18 [==============================] - 1s 16ms/step - loss: 0.0027 - MAE: 0.0400 -

...

17/17 [==============================] - 0s 5ms/step - loss: 0.0045 - MAE: 0.0519 - val_loss: 0.0036 - val_MAE: 0.0468
Epoch 13/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0043 - MAE: 0.0509 - val_loss: 0.0036 - val_MAE: 0.0470
Epoch 14/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0043 - MAE: 0.0510 - val_loss: 0.0036 - val_MAE: 0.0469
Epoch 15/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0045 - MAE: 0.0536 - val_loss: 0.0036 - val_MAE: 0.0469
Epoch 1/15
17/17 [==============================] - 1s 17ms/step - loss: 0.0046 - MAE: 0.0544 - val_loss: 0.0039 - val_MAE: 0.0471
Epoch 2/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0042 - MAE: 0.0513 - val_loss: 0.0039 - val_MAE: 0.0473
Epoch 3/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0043 - MAE: 0.0517 - val_loss: 0.0039 - val_MAE: 0.0475
Epoch 4/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0044 - MAE: 0.0529 - va

17/17 [==============================] - 0s 5ms/step - loss: 0.0033 - MAE: 0.0453 - val_loss: 0.0038 - val_MAE: 0.0475
Epoch 15/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0032 - MAE: 0.0453 - val_loss: 0.0038 - val_MAE: 0.0476
Epoch 1/15
17/17 [==============================] - 1s 17ms/step - loss: 0.0035 - MAE: 0.0458 - val_loss: 0.0029 - val_MAE: 0.0429
Epoch 2/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0034 - MAE: 0.0452 - val_loss: 0.0029 - val_MAE: 0.0427
Epoch 3/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0032 - MAE: 0.0434 - val_loss: 0.0029 - val_MAE: 0.0426
Epoch 4/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0034 - MAE: 0.0449 - val_loss: 0.0029 - val_MAE: 0.0429
Epoch 5/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0031 - MAE: 0.0432 - val_loss: 0.0030 - val_MAE: 0.0433
Epoch 6/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0031 - MAE: 0.0434 - val_

17/17 [==============================] - 1s 17ms/step - loss: 0.0026 - MAE: 0.0401 - val_loss: 0.0020 - val_MAE: 0.0361
Epoch 2/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0026 - MAE: 0.0391 - val_loss: 0.0020 - val_MAE: 0.0359
Epoch 3/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0028 - MAE: 0.0405 - val_loss: 0.0020 - val_MAE: 0.0360
Epoch 4/15
17/17 [==============================] - 0s 8ms/step - loss: 0.0025 - MAE: 0.0380 - val_loss: 0.0020 - val_MAE: 0.0362
Epoch 5/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0028 - MAE: 0.0411 - val_loss: 0.0020 - val_MAE: 0.0362
Epoch 6/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0027 - MAE: 0.0397 - val_loss: 0.0020 - val_MAE: 0.0364
Epoch 7/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0025 - MAE: 0.0383 - val_loss: 0.0021 - val_MAE: 0.0368
Epoch 8/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0025 - MAE: 0.0386 - val_l

17/17 [==============================] - 0s 5ms/step - loss: 0.0022 - MAE: 0.0356 - val_loss: 0.0016 - val_MAE: 0.0316
Epoch 4/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0021 - MAE: 0.0346 - val_loss: 0.0016 - val_MAE: 0.0319
Epoch 5/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0021 - MAE: 0.0350 - val_loss: 0.0016 - val_MAE: 0.0321
Epoch 6/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0022 - MAE: 0.0352 - val_loss: 0.0016 - val_MAE: 0.0323
Epoch 7/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0020 - MAE: 0.0347 - val_loss: 0.0016 - val_MAE: 0.0324
Epoch 8/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0022 - MAE: 0.0367 - val_loss: 0.0017 - val_MAE: 0.0327
Epoch 9/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0021 - MAE: 0.0355 - val_loss: 0.0017 - val_MAE: 0.0329
Epoch 10/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0020 - MAE: 0.0344 - val_l

17/17 [==============================] - 0s 6ms/step - loss: 0.0018 - MAE: 0.0344 - val_loss: 0.0013 - val_MAE: 0.0306
Epoch 6/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0018 - MAE: 0.0333 - val_loss: 0.0013 - val_MAE: 0.0306
Epoch 7/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0018 - MAE: 0.0336 - val_loss: 0.0014 - val_MAE: 0.0309
Epoch 8/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0017 - MAE: 0.0322 - val_loss: 0.0014 - val_MAE: 0.0313
Epoch 9/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0017 - MAE: 0.0324 - val_loss: 0.0014 - val_MAE: 0.0313
Epoch 10/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0018 - MAE: 0.0329 - val_loss: 0.0014 - val_MAE: 0.0311
Epoch 11/15
17/17 [==============================] - 0s 5ms/step - loss: 0.0018 - MAE: 0.0331 - val_loss: 0.0013 - val_MAE: 0.0306
Epoch 12/15
17/17 [==============================] - 0s 6ms/step - loss: 0.0018 - MAE: 0.0327 - val

# Saving the Results

In [8]:
execution_time = end - start

df = pd.DataFrame({"Drugs": drug_names, "pearsonCor": cr , "spearmanCor": spearmanr , "kendallCor": kendalltau , 
                  "RMSE": rmse ,  "MSE": mse , "MAE": mae , "Time": execution_time})
df.to_csv('DL-results/Result_L1000.csv', index=False)